In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#! pip install mosestokenizer

In [0]:
# colab　にmecabを入れる
#!apt install aptitude
#!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
#!pip install mecab-python3==0.7

In [0]:
import sys
import gc
import os
base_dir = "drive/My Drive/jp_en_translation/".replace("/",os.sep)
emb_dir = "drive/My Drive/util_data/".replace("/",os.sep)
sys.path.append("/Users/ueki/Desktop/work/jp_en_translation")
sys.path.append("/Users/ueki/Desktop/JP_EN_Transalation")
sys.path.append(base_dir)
sys.path.append("drive/My Drive/jp_en_translation".replace("/",os.sep))

In [5]:
from models.Seq2Seq_emb import Seq2Seq
from utils.LangEn import LangEn
from utils.LangJa import LangJa
from utils.build_emb import build_en_emb,build_jp_emb
from utils.preprocess import loadLangs
import numpy as np
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
import torch

Using TensorFlow backend.


In [0]:
config={
    "corpus_file":base_dir+"data/jpn.txt",
    "en_col":"description_en",
    "jp_col":"description_jp",
    "SOS_token":1,
    "EOS_token":2,
    "UNK_token":0,
    "max_features":8000,
    "MAX_LENGTH":20,
    "train_size":15000,
    "val_size":100,
    "batch_size":128,
    "epochs":300,
    "maxlen_enc":20,
    "maxlen_dec":20,
    "n_hidden":300,
    "input_dim":5000,
    "jp_voc":8000,
    "en_voc":5000,
    "output_dim":8000,
    "emb_dim":300,
    "use_enc_emb":False,
    "use_dec_emb":False,
    "validation_split":0.1,
    "trained_param_dir":base_dir+"trained_models/1_lstm_ja_en_01.hdf5",
    "translate_length":25,
    "en_W2V_FILE" : emb_dir+"GoogleNews-vectors-negative300.bin.gz",
    "jp_W2V_FILE":emb_dir+"ja_data/ja.bin",
    "src":"en",
    "trg":"jp",
    "learning_rate":0.01,
}

# train

In [7]:
data=loadLangs(config)

reading lines


In [0]:
val_data = data[config["train_size"]:config["train_size"]+config["val_size"]]
data = data[:config["train_size"]]

In [0]:
EN_lang = LangEn(config)
JP_lang = LangJa(config)

In [0]:
for s in data[config["en_col"]]:
    EN_lang.addSentence(s)

In [0]:
for s in data[config["jp_col"]]:
    JP_lang.addSentence(s)

## input の加工

In [0]:
if config["src"]=="jp":
    src_col=config["jp_col"]
    trg_col=config["en_col"]
    Langs={"src":JP_lang,"trg":EN_lang}
else:
    src_col=config["en_col"]
    trg_col=config["jp_col"]
    Langs={"trg":JP_lang,"src":EN_lang}

In [0]:
input_en = data[src_col]

In [0]:
input_source_lang=data[src_col].apply(lambda x:Langs["src"].word2id(x))
input_target_lang=data[trg_col].apply(lambda x:Langs["trg"].word2id(x))

In [0]:
input_source_padded=sequence.pad_sequences(input_source_lang,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")
input_target_padded=sequence.pad_sequences(input_target_lang,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")

In [16]:
input_source_padded[10]

array([50, 56, 57, 42, 58, 59, 60, 27, 61,  5,  2,  0,  0,  0,  0,  0,  0,
        0,  0,  0], dtype=int32)

In [17]:
input_target_padded[10]

array([31, 22, 62, 39, 26, 63, 64,  9, 65, 11, 66,  5,  2,  0,  0,  0,  0,
        0,  0,  0], dtype=int32)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## create embedding

In [19]:
en_emb = build_en_emb(config,EN_lang.word2index)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


voc-> 5000
UNK_rate 0.03740748149629926


In [20]:
ja_emb = build_jp_emb(config,JP_lang.word2index)
ja_emb.shape

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


voc-> 8000
UNK/rate-> 0.6004500562570322


(8000, 300)

In [0]:
if config["src"]=="en":
    emb={"src_emb":en_emb,"trg_emb":ja_emb}
else:
    emb={"src_emb":ja_emb,"trg_emb":en_emb}

In [22]:
print("enc->",emb["trg_emb"].shape)
print("dec->",emb["src_emb"].shape)

enc-> (8000, 300)
dec-> (5000, 300)


In [23]:
trainer = Seq2Seq(config,enc_emb=emb["src_emb"],dec_emb=emb["trg_emb"])

emb torch.Size([5000, 300])
int-> 5000
hid-> 300
emb torch.Size([8000, 300])
int-> 8000
hid-> 300


In [0]:
trainer.trainIters(input_source_padded,input_target_padded)

epoch1 start


# test

In [0]:
val_data_id = val_data[src_col].apply(lambda x:Langs["src"].word2id(x))

In [0]:
val_data_id

In [0]:
ret=[]
for s in val_data_id:
    ret.append(trainer.translate(s))

In [0]:
for src,pred,target in zip(val_data[src_col],ret,val_data[trg_col]):
    print("src->",src)
    print()
    print("pred->"," ".join(Langs["trg"].id2word(pred)))
    print("ans->",target)
    print("------------------")